<a href="https://colab.research.google.com/github/huzaifa003/skin_disease_identification/blob/master/Llama_Chat_Hosted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pypdf

In [2]:
!pip install Flask==3.0.0 pyngrok==7.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
ERROR: Cannot uninstall 'blinker'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [3]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [4]:
!pip install install sentence_transformers

In [5]:
!pip install llama_index==0.9.40

In [6]:
from llama_index import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt


In [7]:

documents=SimpleDirectoryReader("/content/Data").load_data()
documents

[Document(id_='e46c033e-f778-4b09-8642-0e7572ab5dc1', embedding=None, metadata={'page_label': '1', 'file_name': 'Derm.pdf', 'file_path': '/content/Data/Derm.pdf', 'file_type': 'application/pdf', 'file_size': 3171832, 'creation_date': '2024-04-07', 'last_modified_date': '2024-04-07', 'last_accessed_date': '2024-04-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='  British Association of Dermatologists   \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n Dermatology   \n', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='4bf39932-c2b0-4f40-8fe4-e28b327b5fa3', embedding=None, metadata={'page_label': '2', 'file_name': 

In [8]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [11]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-crede

In [12]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

In [14]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)


In [15]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7faaf0ea0eb0>), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7faaf0ea0eb0>, id_func=<function default_id_func at 0x7fabcc768280>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.logger.base.LlamaLogger object at 0x7faacbb6c1f0>, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x7faaf0ea0eb0>)

In [16]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [17]:
query_engine=index.as_query_engine()

In [18]:
response=query_engine.query("What are some common skin diseases?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [19]:
response

Response(response='Some common skin diseases include purpuric eruption, meningococcal septicaemia, disseminated intravascular coagulation, vasculitis, actinic purpura, and history of trauma, malignancy, sepsis, obstetric complications, transfusions, or liver failure. Additionally, there are common sites where skin diseases may occur, such as the extremities, spontaneous bleeding from the ear, nose, and throat, gastrointestinal tract, respiratory tract, or wound site, and dependent areas (e.g. legs, buttocks, flanks). Lesions may appear as petechiae, ecchymoses, haemorrhagic bullae, and/or tissue necrosis, and may be painful or non-palpable. Associated features may include systemic illness, and possible investigations may include bloods, lumbar puncture, and skin biopsy. Management may involve antibiotics, transfusions, anticoagulants, steroids, and immunosuppressants, depending on the underlying cause and', source_nodes=[NodeWithScore(node=TextNode(id_='09a117f3-cf24-4d6b-93f4-eb4eb6e2

In [20]:
print(response)

Some common skin diseases include purpuric eruption, meningococcal septicaemia, disseminated intravascular coagulation, vasculitis, actinic purpura, and history of trauma, malignancy, sepsis, obstetric complications, transfusions, or liver failure. Additionally, there are common sites where skin diseases may occur, such as the extremities, spontaneous bleeding from the ear, nose, and throat, gastrointestinal tract, respiratory tract, or wound site, and dependent areas (e.g. legs, buttocks, flanks). Lesions may appear as petechiae, ecchymoses, haemorrhagic bullae, and/or tissue necrosis, and may be painful or non-palpable. Associated features may include systemic illness, and possible investigations may include bloods, lumbar puncture, and skin biopsy. Management may involve antibiotics, transfusions, anticoagulants, steroids, and immunosuppressants, depending on the underlying cause and


In [21]:
%%time
response=query_engine.query("What is Eczema?")

CPU times: user 45.9 s, sys: 62.4 ms, total: 45.9 s
Wall time: 46.3 s


In [22]:
print(response)

Eczema, also known as atopic dermatitis, is a chronic inflammatory skin condition characterized by dry, itchy, and scaly patches on the skin. It is common in children but can also affect adults. The exact cause of eczema is not fully understood, but it is believed to involve a combination of genetic and environmental factors. Eczema can occur anywhere on the body but is most commonly found on the face, hands, and feet. It can also lead to pigmentary changes, such as hypopigmentation or hyperpigmentation, and nail changes, such as pitting and ridging. Treatment for eczema typically involves a combination of topical and systemic medications, including corticosteroids, immunomodulators, and antihistamines, as well as lifestyle modifications such as avoiding exacerbating factors and using emollients to moisturize the skin.


In [23]:
%%time
response=query_engine.query("What are some treatments of Eczema?")

CPU times: user 43.8 s, sys: 83.5 ms, total: 43.8 s
Wall time: 44.2 s


In [24]:
print(response)

Based on the information provided in the context, some treatments for eczema include:

1. Topical steroids for active areas
2. Topical immunomodulators (e.g. tacrolimus, pimecrolimus) for maintenance therapy as steroid-sparing agents
3. Oral antihistamines for symptomatic relief
4. Antibiotics (e.g. flucloxacillin) for secondary bacterial infections
5. Antivirals (e.g. aciclovir) for secondary herpes infection.

It is important to note that the most appropriate treatment for eczema will depend on the severity and location of the condition, as well as the individual patient's needs and medical history. A healthcare professional should be consulted for proper diagnosis and treatment.


In [35]:
print(str(response))

Based on the information provided in the context, some treatments for eczema include:

1. Topical steroids for active areas
2. Topical immunomodulators (e.g. tacrolimus, pimecrolimus) for maintenance therapy as steroid-sparing agents
3. Oral antihistamines for symptomatic relief
4. Antibiotics (e.g. flucloxacillin) for secondary bacterial infections
5. Antivirals (e.g. aciclovir) for secondary herpes infection.

It is important to note that the most appropriate treatment for eczema will depend on the severity and location of the condition, as well as the individual patient's needs and medical history. A healthcare professional should be consulted for proper diagnosis and treatment.


In [25]:
ngrok_key = "2YEEfAspjAhS7hCF9XvndSLiUQP_7fgVsnfKCFY7D66LJGKxy"
port = 5000

In [26]:
from pyngrok import ngrok

In [41]:
ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

'https://87d7-34-72-58-136.ngrok-free.app'

In [42]:

from flask import Flask, request

app = Flask(__name__)

@app.route("/derm", methods=['POST'])
def derm():
    # Assuming the data is sent as JSON, with a key named 'query'
    data = request.json
    query = data.get('query', 'Name some common skin disease')

    response = query_engine.query(query)  # Use the query from the user
    return str(response)

if __name__ == '__main__':
    app.run(port = port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [07/Apr/2024 22:34:44] "POST /derm HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Apr/2024 22:35:57] "POST /derm HTTP/1.1" 200 -
